In [1]:
import numpy as np
import pandas as pd
import typing

from scipy.stats import norm

# The Plan
A mixture of discrete time physical simulation and statistical simulation will be used. The problem is decomposed into multiple layers:
- atmospherical virual pressure: a proxy to measure how much visrus is present in the environment. The higher the pressure, the more likely an individual gets infected.
- transport layer: this layer dictates how individuals move around within the space, a matrix of transition probability between key locations. Individuals movements are assumed to be following a normal distribution around each key location
- population layer: this layer keeps track of individuals status including location, a list of individuals
- immigration layer: how new individuals are introduced into the population layer
- medical resource layer: it indicates how much resources is available to test and care for potential patients. This will also affect our ability of observe the population. A rough measure is number of people per m^2

# Parameters
## Parameters of interest
- Infectious rate: temperature
- Asymptomatic time: age
- Death rate: age
- Infectious radius: an arbitary constant
- Virus life span: how long the virus stays infectious in the environment, temperature related

## Other variables
- Spatial information: cordinates
- Mobility: change of cordinates
- Population


# Setting up

## Constants

In [2]:
X_BOUNDARY = (0, 300000) #in meters
Y_BOUNDARY = (0, 800000) #in meters
POPULATION = 67886011

AVERAGE_TEMPRATURE = 15.0
VIRUS_LIFE_EXPECTANCY = 4.0 #4 hours

## Environment

In [3]:
#Using real data for modelling UK population concentration spread
city_population = pd.read_csv("city_population.csv")
city_population.columns = columns=["city", "population", "location"]
city_population["proportion"] = city_population.population / sum(city_population.population)
population_distribution = city_population["proportion"]

In [4]:
#Using real data for modelling UK age group
age_groups = pd.read_csv("uk_age_groups.csv").dropna()
age_groups = age_groups.drop(["Ethnicity", "All ages"], axis=1)

age_groups = age_groups.applymap(lambda x: int(x.replace(",", "")))
age_groups["Age 5 to 9"] = age_groups["Age 5 to 7"] + age_groups["Age 8 to 9"]
age_groups["Age 15 to 19"] = age_groups["Age 15"] + age_groups["Age 16 to 17"] + age_groups["Age 18 to 19"]

age_groups = age_groups.drop(columns=["Age 5 to 7", "Age 8 to 9", "Age 15", "Age 16 to 17", "Age 18 to 19"], axis=1)
age_groups = pd.DataFrame.sum(age_groups, axis=0).to_frame()

age_groups = age_groups.set_index(np.array([4, 14, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, 85, 9, 19]))
age_groups = age_groups.sort_index()

age_groups.columns = ["population"]
age_groups["age_distribution"] = age_groups["population"] / pd.DataFrame.sum(age_groups["population"])
age_distribution = age_groups["age_distribution"]

## Defining a person

In [7]:
class Location():
    def __init__(self, x: float , y: float):
        self.x = x
        self.y = y
    
class Individual():
    def __init__(self, location: Location=Location(0,0), age: int=40, is_infected: bool=False, 
                 infection_date: int=None, has_symptom: bool=False, symptom_date: int=None, is_dead: bool=False, 
                 death_date: int=None, vulnerability: float=0.5, mobility: float=0.5, immunity: float=0.5, 
                 infection_radius:float=2):
        self.location = location
        self.age = age
        self.is_infected = is_infected
        self.infection_date = infection_date
        self.has_symptom = has_symptom #has symptoms
        self.symptom_date = symptom_date #when symptom menifested
        self.is_dead = is_dead #i.e. not alive
        self.death_date = death_date # when one died
        self.vulnerability = vulnerability #interacting with death
        self.mobility = mobility #cross region mobility
        self.immunity = immunity #interacting with infection layer
        self.infection_radius = infection_radius #effective infectious radius, a part of the smoothing parameter
    
    def cal_distance(location: Location):
        return sqrt((self.x - location.x)**2 + (self.y - location.y)**2)
    
    def get_infection_pressure(self,
                               location:Location,
                               infectious_rate:float):
        #assumming a normally distributed infection contribution.
        mu = cal_distance(location)
        return infectious_rate * norm.pdf(x=mu, scale=self.infection_radius*3)
    
    def get_individual_array(self):
        return np.array(self.location.x, self.location.y, self.ago, self.is_infected,
                        self.infection_date, self.has_symptom, self.symptom_date,
                        self.is_dead, self.death_date, self.vulnerability, self.mobility,
                        self.immunity, self.infection_radius)
        

class Population():
    def __init__(self, population_location:Location, population_size: int, individuals: [Individual]):
        self.population_location = population_location
        self.population_size = population_size
        self.individuals_list = individuals
        self.individuals_array = None
        
    def get_infected_count(self):
        infected_count = 0
        for individual in self.individuals_list:
            infected_count += individual.is_infected
        return infected_count
    
    def get_has_symptom_count(self):
        has_symptom_count = 0
        for individual in self.individuals_list:
            has_symptom_count += individual.has_symptom
        return has_symptom_count
    
    def get_death_count(self):
        death_count = 0
        for individual in self.individuals_list:
            death_count += individual.is_dead
        return death_count
    
    

In [6]:
%time x = np.random.uniform(0,1, 1000000)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 35.9 ms


In [ ]:
%time len(x[np.logical_and((x > 0.3), (x < 0.7))])